Only for scrapping a page

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the webpage to scrape
url = "http://5000best.com/websites/"

# Fetch and parse the webpage
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to fetch the webpage: {response.status_code}")
else:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the table with id "ttable"
    table = soup.find('table', {'id': 'ttable'})
    if not table:
        print("No table with id 'ttable' found.")
    else:
        # Find all rows within the table body
        rows = table.find('tbody').find_all('tr')
        print(f"Number of rows found: {len(rows)}")

        # Prepare the CSV file
        with open("websites_data.csv", mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(["Category", "URL", "Description"])  # Write the header row

            # Iterate over each row to extract data
            for row in rows:
                columns = row.find_all('td')
                if len(columns) >= 8:  # Ensure there are enough columns
                    category = columns[2].text.strip()  # Column 3 for Category
                    url = columns[4].a['href'] if columns[4].find('a') else columns[4].text.strip()  # Column 5 for URL
                    description = columns[7].text.strip()  # Column 8 for Description
                    writer.writerow([category, url, description])

        print("Data successfully scraped and saved to 'websites_data.csv'.")


Number of rows found: 100
Data successfully scraped and saved to 'websites_data.csv'.


For scrapping all page from the website

In [ ]:
#this actually make the dataset for website classification
import requests
from bs4 import BeautifulSoup
import csv
import time

# Base URL of the website
base_url = "http://5000best.com"

# Output CSV file
output_file = "websites_data.csv"

# Prepare the CSV file
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "URL", "Description"])  # Write header row

    # Start with the first page
    next_page_url = "/websites/"
    retries = 3  # Maximum retries for each page

    while next_page_url:
        url = f"{base_url}{next_page_url}"
        print(f"Scraping page: {url}")

        for attempt in range(retries):
            try:
                # Fetch the page
                # Need this timeout as the webpage reject if it gets so many request continuously
                response = requests.get(url, timeout=10)
                response.raise_for_status()  # Raise an error for HTTP codes >= 400
                break  # Exit retry loop if request is successful
            except requests.exceptions.RequestException as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt < retries - 1:
                    time.sleep(5)  # Wait before retrying
                else:
                    print(f"Failed to fetch {url} after {retries} attempts.")
                    next_page_url = None  # Stop further scraping
                    break
        else:
            continue  # Move to the next iteration of the while loop

        # Parse the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table with id "ttable"
        table = soup.find('table', {'id': 'ttable'})
        if not table:
            print("No table found on this page.")
            break

        # Extract rows from the table
        rows = table.find('tbody').find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) >= 8:  # Ensure there are enough columns
                category = columns[2].text.strip()
                url = columns[4].a['href'] if columns[4].find('a') else columns[4].text.strip()
                description = columns[7].text.strip()
                writer.writerow([category, url, description])

        # Find the "Next" page link
        pagination = soup.find('div', {'id': 'dpages'})
        if pagination:
            current_page_num = int(next_page_url.split('/')[-2]) if next_page_url.split('/')[-2].isdigit() else 1
            next_link = pagination.find('a', text=f"[{current_page_num + 1}]")
            if next_link:
                next_page_url = next_link['href']
            else:
                next_page_url = None
        else:
            next_page_url = None

        # Delay between requests
        time.sleep(5)

print(f"Data successfully scraped and saved to '{output_file}'.")


Scraping page: http://5000best.com/websites/


<ipython-input-23-f5a335e19207>:65: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_link = pagination.find('a', text=f"[{current_page_num + 1}]")


Scraping page: http://5000best.com/websites/2/
Scraping page: http://5000best.com/websites/3/
Scraping page: http://5000best.com/websites/4/
Scraping page: http://5000best.com/websites/5/
Scraping page: http://5000best.com/websites/6/
Scraping page: http://5000best.com/websites/7/
Attempt 1 failed: HTTPConnectionPool(host='5000best.com', port=80): Max retries exceeded with url: /websites/7/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7deb1a33ef50>: Failed to establish a new connection: [Errno 111] Connection refused'))
Scraping page: http://5000best.com/websites/8/
Scraping page: http://5000best.com/websites/9/
Scraping page: http://5000best.com/websites/10/
Scraping page: http://5000best.com/websites/11/
Scraping page: http://5000best.com/websites/12/
Scraping page: http://5000best.com/websites/13/
Scraping page: http://5000best.com/websites/14/
Scraping page: http://5000best.com/websites/15/
Scraping page: http://5000best.com/websites/16/
Attempt 1 f

In [ ]:
import pandas as pd

# Replace 'websites_data.csv' with the correct path to your CSV file
file_path = "websites_data.csv"

# Read the CSV file
csv_data = pd.read_csv(file_path)

# Display the content of the CSV file
print(csv_data)

        Category                            URL  \
0      Discovery              http://google.com   
1         People            http://facebook.com   
2         Videos             http://youtube.com   
3        Portals               http://yahoo.com   
4     Discussion             http://twitter.com   
...          ...                            ...   
4995         Web         http://directtrack.com   
4996        Porn               http://cliti.com   
4997         Web           http://ispionage.com   
4998      Videos            http://videobam.com   
4999   Discovery  http://5000best.com/websites/   

                                            Description  
0     Google - Search the world's information, inclu...  
1     Welcome to Facebook - Facebook helps you conne...  
2     YouTube - Share your videos with friends, fami...  
3                                                Yahoo!  
4     Twitter - Instantly connect to what's most imp...  
...                                    

In [ ]:
import pandas as pd

# Load the scraped CSV file
file_path = "websites_data.csv"

# Read the CSV file
try:
    data = pd.read_csv(file_path)

    # Count the number of websites per category
    category_counts = data['Category'].value_counts()

    # Display the category counts
    print("Category-wise Website Counts:")
    print(category_counts)

    # Save the results to a CSV file
    category_counts.to_csv("category_counts.csv", header=["Count"])
    print("Category counts have been saved to 'category_counts.csv'.")
except FileNotFoundError:
    print("The file 'websites_data.csv' was not found. Please ensure the file exists and try again.")


Category-wise Website Counts:
Category
Web              786
Commerce         405
News             312
Games            247
Technology       225
Education        221
Tools            207
Porn             201
Services         199
Style            172
Files            168
Pictures         161
Business         155
Articles         146
Health           138
Travel           135
Entertainment    117
Sport            114
Music            106
Discussion        96
Videos            82
Organizations     76
TV                70
People            68
Discovery         65
Cars              61
Humor             60
Jobs              59
Books             48
Movies            47
Science           31
Portals           22
Name: count, dtype: int64
Category counts have been saved to 'category_counts.csv'.
